# Gauss elimination and LU decomposition

In [1]:
import numpy as np

### Utility

In [2]:
def generate_matrix(n_cols: int, n_rows: int):
    # Uniform używa rozkładu jednostajnego ciągłego
    return np.random.uniform(low=0.1, high=0.01, size=(n_rows, n_cols))

def generate_vector(n: int):
    return np.random.uniform(low=0.1, high=0.01, size=(n,))

In [3]:
generate_matrix(5, 5), generate_vector(5)

(array([[0.02461908, 0.02376166, 0.03750922, 0.08450792, 0.06400768],
        [0.06116688, 0.05988596, 0.03253187, 0.07311733, 0.01685579],
        [0.03155311, 0.01442802, 0.02966505, 0.03741621, 0.08571038],
        [0.03197542, 0.02755534, 0.09268297, 0.08691851, 0.01945102],
        [0.01582654, 0.01613453, 0.02744073, 0.03698507, 0.09684359]]),
 array([0.04915923, 0.01475601, 0.03896281, 0.0343995 , 0.0305812 ]))

Test

### Gauss elimination

In [4]:
def gauss_elimination(A, b):
    n = len(A)
    
    for i in range(n):
        divisor = A[i, i]
        A[i] /= divisor
        b[i] /= divisor
        
        for j in range(i + 1, n):
            multiplier = A[j, i]
            A[j] -= multiplier * A[i]
            b[j] -= multiplier * b[i]


    # iterate columns for last to first
    for i in range(n - 1, -1, -1):
        for j in range(i):
            multiplier = A[j, i]
            A[j] -= multiplier * A[i]
            b[j] -= multiplier * b[i]
    
    return b


def gauss_elimination_pivoting(A, b):
    n = len(A)
    pivots = []
    
    for i in range(n):
        pivot_row = np.argmax(np.abs(A[i:, i])) + i
        if pivot_row != i:
            pivots.append((i, pivot_row))
            A[[i, pivot_row]] = A[[pivot_row, i]]
            b[[i, pivot_row]] = b[[pivot_row, i]]
        
        divisor = A[i, i]
        A[i] /= divisor
        b[i] /= divisor
        
        for j in range(i + 1, n):
            multiplier = A[j, i]
            A[j] -= multiplier * A[i]
            b[j] -= multiplier * b[i]

    # iterate columns for last to first
    for i in range(n - 1, -1, -1):
        for j in range(i):
            multiplier = A[j, i]
            A[j] -= multiplier * A[i]
            b[j] -= multiplier * b[i]

    for i, pivot_row in pivots[::-1]: 
        A[[i, pivot_row]] = A[[pivot_row, i]]
        b[[i, pivot_row]] = b[[pivot_row, i]]

    return b

In [5]:
from numpy.linalg import solve

n = 17 + 9

A = generate_matrix(n, n)
b = generate_vector(n)

result_gauss = gauss_elimination(A, b)
result_gauss_pivot = gauss_elimination_pivoting(A, b)
result_numpy = solve(A, b)

print(result_gauss)
print(result_gauss_pivot)
print(result_numpy)

print(f"Are the results close: {np.allclose(result_gauss, result_numpy)}")
print(f"Are the results close: {np.allclose(result_gauss_pivot, result_numpy)}")

[-0.5848295   0.24881951 -0.27610172  0.74305406 -0.34318177 -0.20019773
 -0.35006181  0.41637584  0.10507856 -0.80577365  0.93977864 -0.05928053
  0.61133359 -0.10757861 -0.61382435 -0.35987023 -0.43653054 -0.47560416
 -0.22155523  0.04691706  0.79338809  1.33174254  0.20792416  0.36434841
  0.05551713  0.11975954]
[-0.5848295   0.24881951 -0.27610172  0.74305406 -0.34318177 -0.20019773
 -0.35006181  0.41637584  0.10507856 -0.80577365  0.93977864 -0.05928053
  0.61133359 -0.10757861 -0.61382435 -0.35987023 -0.43653054 -0.47560416
 -0.22155523  0.04691706  0.79338809  1.33174254  0.20792416  0.36434841
  0.05551713  0.11975954]
[-0.5848295   0.24881951 -0.27610172  0.74305406 -0.34318177 -0.20019773
 -0.35006181  0.41637584  0.10507856 -0.80577365  0.93977864 -0.05928053
  0.61133359 -0.10757861 -0.61382435 -0.35987023 -0.43653054 -0.47560416
 -0.22155523  0.04691706  0.79338809  1.33174254  0.20792416  0.36434841
  0.05551713  0.11975954]
Are the results close: True
Are the results cl

### LU decomposition

In [43]:
def lu_decomposition(A):
    n = len(A)
    L = np.eye(n)
    U = np.zeros_like(A)
    
    for i in range(n):
        # Update U matrix
        for j in range(i, n):
            U[i, j] = A[i, j] - np.dot(L[i, :i], U[:i, j])
        
        # Update L matrix
        for j in range(i + 1, n):
            L[j, i] = (A[j, i] - np.dot(L[j, :i], U[:i, i])) / U[i, i]
    
    return L, U

def lu_decomposition_pivoting(A):
    n = len(A)
    L = np.eye(n)
    U = A.copy()
    P = np.eye(n) 
    
    for i in range(n):
        # Partial pivoting: find the row with the largest element in the current column
        pivot_row = np.argmax(np.abs(U[i:, i])) + i
        if pivot_row != i:
            # Swap rows in U and L
            U[[i, pivot_row]] = U[[pivot_row, i]]
            L[[i, pivot_row]] = L[[pivot_row, i]]
            # Update permutation matrix P
            P[[i, pivot_row]] = P[[pivot_row, i]]

        for j in range(i + 1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, i:] -= L[j, i] * U[i, i:]
    
        print(L)
        print(U)
        print()

    # THIS CALCULATES P @ A = L @ U, but we ned A = P @ L @ U to ocmpare with scipy
    # INVERSE OF PERMUTATION MATRIX IS ITS TRANSPOSE
    return P.T, L, U

# Example usage:
A = generate_matrix(3, 3)

P, L, U = lu_decomposition_pivoting(A)

print(P)
print(L)
print(U)

print(A)
print(P @ L @ U)

# L, U = lu_decomposition(A)
np.allclose(P @ L @ U, A)

[[0.         0.         1.        ]
 [0.3913228  1.         0.        ]
 [0.76367949 0.         0.        ]]
[[0.07243582 0.05167551 0.04367147]
 [0.         0.0529795  0.02211355]
 [0.         0.02261431 0.01293263]]

[[0.         0.         1.        ]
 [0.3913228  1.         0.        ]
 [0.76367949 0.42685027 0.        ]]
[[0.07243582 0.05167551 0.04367147]
 [0.         0.0529795  0.02211355]
 [0.         0.         0.00349345]]

[[0.         0.         1.        ]
 [0.3913228  1.         0.        ]
 [0.76367949 0.42685027 0.        ]]
[[0.07243582 0.05167551 0.04367147]
 [0.         0.0529795  0.02211355]
 [0.         0.         0.00349345]]

[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
[[0.         0.         1.        ]
 [0.3913228  1.         0.        ]
 [0.76367949 0.42685027 0.        ]]
[[0.07243582 0.05167551 0.04367147]
 [0.         0.0529795  0.02211355]
 [0.         0.         0.00349345]]
[[0.05531775 0.06207784 0.04628363]
 [0.02834579 0.07320131 0.03920319]
 [0.07243582 0.0

False

In [38]:
from scipy.linalg import lu

n = 17 + 9
n = 3

A = generate_matrix(n, n)

L, U = lu_decomposition(A)
P_piv, L_piv, U_piv = lu_decomposition_pivoting(A)
# result_gauss_pivot = gauss_elimination_pivoting(A, b)
sp_P, sp_L, sp_U = lu(A)

print(L)
print(sp_L)
print(U)
print(sp_U)
# print(result_gauss_pivot)
print(sp_P)
print(f"Are the results close: {np.allclose(sp_L, L)}")
# print(f"Are the results close: {np.allclose(result_gauss_pivot, result_numpy)}")

[[1.         0.         0.        ]
 [6.61474742 1.         0.        ]
 [5.88589816 0.8608674  1.        ]]
[[1.         0.         0.        ]
 [0.15117735 1.         0.        ]
 [0.8898145  0.19147777 1.        ]]
[[ 0.01097053  0.08371472  0.0969254 ]
 [ 0.         -0.53353498 -0.62982996]
 [ 0.          0.          0.04220958]]
[[0.07256728 0.02021676 0.01130706]
 [0.         0.08065841 0.09521603]
 [0.         0.         0.04220958]]
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
Are the results close: False
